In [ ]:
#Preparação do ambiente LINUX

!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://cli.github.com/packages stable InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
openjdk versio

In [ ]:
!pip install pyspark

In [ ]:
# Dataset NASA Logs
!git clone https://github.com/leonardoamorim/aulapython.git

fatal: destination path 'aulapython' already exists and is not an empty directory.


In [ ]:
! ls aulapython

 airlinedelaycauses_DelayedFlights.csv.part01.rar
 airlinedelaycauses_DelayedFlights.csv.part02.rar
 airlinedelaycauses_DelayedFlights.csv.part03.rar
 class_summary_statistics_asyncio.py
 class_summary_statistics_numba.py
 class_summary_statistics.py
 colaboradores.csv
 colaboradores_data_missing.csv
 exercicio_01.py
 funcionarios.json
'HPC_com_Python_para_Big_Data_lab_vecAdd_PyCUDA (1).ipynb'
 IRIS.csv
 iris.data
 KNN-RegressaoLogistica.ipynb
 NASA_access_log_Aug95
 NASA_access_log_Jul95
'Operações avançadas com DataFrame usando PySpark.ipynb'
'Operacoes com dados faltantes.ipynb'
'Operacoes com Datas e Timestamps.ipynb'
'Pandas - Intro.ipynb'
 programa1.py
 programa2.py
 programa3.py
'Pyspark - Operações Básicas com DF.ipynb'
' Python para Big Data - Introdução a Programação Funcional com Python Funções Lambda em Python.ipynb'
'RAPIDS(1).ipynb'
 Regressão_linear_com_Python.ipynb
 Regressão_logística_com_Python.ipynb
 titanic_test.csv
 titanic_train.csv
 Tutorial_Cuda_no_Colab.ipynb
 

In [ ]:
# Descompactar esses arquivos

! gunzip aulapython/NASA_access_log_Jul95.gz
! gunzip aulapython/NASA_access_log_Aug95.gz

gzip: aulapython/NASA_access_log_Jul95.gz: No such file or directory
gzip: aulapython/NASA_access_log_Aug95.gz: No such file or directory


In [ ]:
# Quantidade de disco usado
! du -h aulapython/NASA_access_log_Jul95

196M	aulapython/NASA_access_log_Jul95


In [ ]:
! du -h aulapython/NASA_access_log_Aug95

161M	aulapython/NASA_access_log_Aug95


In [ ]:
# Verificação da quantidade de linhas do arquivo
! wc -l aulapython/NASA_access_log_Jul95
! wc -l aulapython/NASA_access_log_Jul95

1891714 aulapython/NASA_access_log_Jul95
1891714 aulapython/NASA_access_log_Jul95


# Conjunto de dados distribuídos resiliente (RDD)


In [ ]:
from pyspark import SparkConf, SparkContext #instanciar o driver
from operator import add

In [ ]:
# setMaster("local") : definição do cluster
# setAppName("Exercicio") : define o nome da execução
# definiu 5 gigas de memória para o processamento dessa aplicação
configuracao = (SparkConf().setMaster("local").setAppName("Exercicio").set("spark.executor.memory","5g"))

In [ ]:
sc = SparkContext(conf = configuracao) # objeto

In [ ]:
# Criação de 2 RDDs , carregando esses dados do tipo texto em memória
julho = sc.textFile('aulapython/NASA_access_log_Jul95')
agosto = sc.textFile('aulapython/NASA_access_log_Aug95')

In [ ]:
julho = julho.cache()
agosto = agosto.cache()

In [ ]:
# imprime as 3 primeiras linhas do cabeçalho desse arquivo
! head -n3 aulapython/NASA_access_log_Jul95

199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245
unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985
199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085


In [ ]:
exemplo = '199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245'

In [ ]:
operacao = exemplo.split() # separando pelo padrão da função que é ' '
print(operacao)

['199.72.81.55', '-', '-', '[01/Jul/1995:00:00:01', '-0400]', '"GET', '/history/apollo/', 'HTTP/1.0"', '200', '6245']


In [ ]:
print(operacao[0])

199.72.81.55


In [ ]:
# Pega a lista dos hosts das 5 primeiras linhas
julho.map(lambda line: line.split()[0]).take(5)

['199.72.81.55',
 'unicomp6.unicomp.net',
 '199.120.110.21',
 'burger.letters.com',
 '199.120.110.21']

In [ ]:
contagem_julho = julho.map(lambda line: line.split()[0]).distinct().count()
print("Numero de hosts distintos no mes de julho:", contagem_julho)

Numero de hosts distintos no mes de julho: 81983


In [ ]:
contagem_agosto = agosto.map(lambda line: line.split()[0]).distinct().count()
print("Numero de hosts distintos no mes de agosto:", contagem_agosto)

Numero de hosts distintos no mes de agosto: 75060


In [ ]:
def codigo404(linha):
  try:
    codigohttp = linha.split()[-2]
    if codigohttp == '404':
      return True
  except:
    pass
  return False

In [ ]:
errors404_julho = julho.filter(codigo404).cache() # guardar em cache
errors404_agosto = agosto.filter(lambda linha: linha.split()[-2] == '404').cache() # guardar em cache

In [ ]:
print(errors404_julho.count())
print(errors404_agosto.count())

10845
10056


In [ ]:
print(exemplo.split('"')[1].split()[1])

/history/apollo/


In [ ]:
urls = errors404_julho.map( lambda linha: linha.split('"')[1].split()[1] )
counts = urls.map( lambda url: (url, 1) ).reduceByKey(add) # add é um operador de soma, conta a quantidade de vezes que o url apareceu em julho

In [ ]:
counts.take(10)

[('/shuttle/missions/technology/sts-newsref/stsref-toc.html', 107),
 ('/history/apollo-13/apollo-13.html', 73),
 ('/history/apollo/pad-abort-test-1/images/', 47),
 ('/de/systems.html', 26),
 ('/Titles?qt=nasa', 2),
 ('/shuttle/resources/orbiters/atlantis.gif', 232),
 ('/shuttle/missions/sts-71/ksc-status-06-05-95.txt', 1),
 ('/.pub.win3.winvn', 4),
 ('/software/winvn/winvn.html.', 33),
 ('/lc39a.gif', 1)]

In [ ]:
def top5_hosts404(rdd):
  urls = rdd.map( lambda linha: linha.split('"')[1].split()[1] )
  counts = urls.map( lambda url: (url, 1) ).reduceByKey(add)
  top5 = counts.sortBy( lambda par: -par[1]).take(5)

  return top5

In [ ]:
top5_hosts404(errors404_julho)

[('/pub/winvn/readme.txt', 667),
 ('/pub/winvn/release.txt', 547),
 ('/history/apollo/apollo-13.html', 286),
 ('/shuttle/resources/orbiters/atlantis.gif', 232),
 ('/history/apollo/a-001/a-001-patch-small.gif', 230)]

In [ ]:
top5_hosts404(errors404_agosto)

[('/pub/winvn/readme.txt', 1337),
 ('/pub/winvn/release.txt', 1185),
 ('/shuttle/missions/STS-69/mission-STS-69.html', 683),
 ('/images/nasa-logo.gif', 319),
 ('/shuttle/missions/sts-68/ksc-upclose.gif', 253)]

In [ ]:
# diferença entre reduce e reduceByKey ?